In [ ]:
from deltalake import DeltaTable, write_deltalake
from pathlib import Path
import pandas as pd
import polars as pl
from tpot import TPOTClassifier, TPOTRegressor
from sklearn.model_selection import train_test_split

In [2]:
input_data_path = r"../data/delta"

In [3]:
dt = DeltaTable(input_data_path)
# dict of partitions
entity_partitions = dt.partitions()
sample_fraction = 1.0

In [4]:
entity_partitions = [{'entity': 'entity_18'},
 {'entity': 'entity_13'},
 {'entity': 'entity_7'},
]
entity_partition = "entity_1"
entity_col = 'entity'

In [5]:
# tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
tpot = TPOTClassifier(generations=2, population_size=3, verbosity=2, random_state=42)

In [ ]:
entity_pldf = pl.read_delta(input_data_path, pyarrow_options={"partitions": [("entity", "=", f"{entity_partition}")]})
entity_pdf = entity_pldf.sample(fraction=sample_fraction).to_pandas()

features = [s for s in entity_pdf.columns.to_list() if s.isdigit()]
target = 'target'

# prepare data
X_train, X_test, y_train, y_test = train_test_split(
    entity_pdf[features], entity_pdf[target], train_size=0.75, test_size=0.25, random_state=42
)

entity_pdf.shape, X_train.shape, y_train.shape

In [ ]:
tpot.fit(X_train, y_train)

In [ ]:
tpot.score(X_test, y_test)